# Vlasov-Poisson Solver Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/user/vlasov_poisson_solver/blob/main/demo/demo.ipynb)

This notebook demonstrates the usage of the `vlasov_poisson_solver` package.

In [ ]:
# Install the package (uncomment if running in Colab)
# !pip install git+https://github.com/user/vlasov_poisson_solver.git

In [ ]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from vlasov_poisson_solver import VlasovPoissonSolver, DomainConfig, PhysicsConfig, SolverConfig, create_mesh

In [ ]:
# Configuration
domain_config = DomainConfig(
    x_min=0.0, x_max=4*jnp.pi,
    y_min=0.0, y_max=4*jnp.pi,
    vx_min=-6.0, vx_max=6.0,
    vy_min=-6.0, vy_max=6.0,
    Nx=64, Ny=64, Nvx=64, Nvy=64
)
physics_config = PhysicsConfig(dt=0.1, final_time=1.0)
solver_config = SolverConfig(interpolation_order=1)

In [ ]:
# Initialize Solver
mesh = create_mesh((1, 1), ('x', 'y'))
solver = VlasovPoissonSolver(domain_config, physics_config, solver_config, mesh=mesh)

In [ ]:
# Initial Condition: Landau Damping
alpha = 0.01
kx = 0.5
ky = 0.5

X, Y = jnp.meshgrid(solver.x, solver.y, indexing='ij')
VX, VY = jnp.meshgrid(solver.vx, solver.vy, indexing='ij')

X = X[:, :, None, None]
Y = Y[:, :, None, None]
VX = VX[None, None, :, :]
VY = VY[None, None, :, :]

spatial_perturbation = 1.0 + alpha * (jnp.cos(kx * X) + jnp.cos(ky * Y))
velocity_dist = (1.0 / (2 * jnp.pi)) * jnp.exp(-(VX**2 + VY**2) / 2)

f = spatial_perturbation * velocity_dist

In [ ]:
# Run Simulation
steps = 10
for i in range(steps):
    f = solver.step(f, i * physics_config.dt)
    print(f"Step {i+1}/{steps} completed")

In [ ]:
# Plot Density
rho = jnp.sum(f, axis=(2, 3)) * domain_config.dvx * domain_config.dvy
plt.imshow(rho.T, origin='lower', extent=[0, 4*jnp.pi, 0, 4*jnp.pi])
plt.colorbar(label='Density')
plt.title('Charge Density')
plt.show()